In [18]:
import awk_data
import tensorflow as tf
import argparse 
from collections import defaultdict
import os, json
import importlib.util
from time import time
import numpy as np
import plot_loss

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
baseModelDatapath = "/eos/user/d/dvalsecc/www/ECAL/Clustering/DeepCluster/models_archive/gcn_models/ACAT2022/best_models_archive/tests_simpler_rechits/run_20_datasetv10_verysmall_nconv3/"

In [20]:
config = json.load(open("training_config.json"))

In [21]:
## Loading the datasets
print(">>> Loading datasets")

train_ds = awk_data.load_dataset(awk_data.LoaderConfig(**config["dataset_conf"]["training"]))
test_ds = awk_data.load_dataset(awk_data.LoaderConfig(**config["dataset_conf"]["validation"]))
# Create training and validation
ds_train = train_ds.prefetch(tf.data.AUTOTUNE).repeat(config['nepochs'])
ds_test  = test_ds.prefetch(tf.data.AUTOTUNE).repeat(config['nepochs'])

>>> Loading datasets


In [22]:
############### 
# Loading the model file
 # Load model modules
spec = importlib.util.spec_from_file_location("model", "model_simpler_rechits.py")
model_lib = importlib.util.module_from_spec(spec)
spec.loader.exec_module(model_lib)

In [23]:
# Build the model with all the configs
model = model_lib.DeepClusterGN(**config)
opt = tf.keras.optimizers.Adam(learning_rate=config['lr'])
model.compile(optimizer=opt)
model.set_metrics()

In [ ]:
for X, y, w  in ds_train:
    # Load the model
    ypred = model(X, training=False)
    #l = custom_loss(y, ypred)
    break